In [1]:
import math

import findspark
findspark.init()

import pyspark
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating


import collections
import time
import sys

import nltk
import sklearn
import numpy
import pandas as pd

sc = pyspark.SparkContext("local", "Recommendation system")

training_data = sc.textFile("tempe_train.csv")
test_data = sc.textFile("tempe_test.csv")




In [2]:
from sklearn.preprocessing import StandardScaler  

In [4]:
header_new = test_data.first()

test_data.filter(lambda x: x!=header_new).take(4)

[u'-3gIDSqZ04FROn3du4CK2A,-xIrVu_7prj_0GDl1tds1w,5',
 u'-3gIDSqZ04FROn3du4CK2A,Bjn-GQg6hoUOkew7tTPcLA,5',
 u'-3gIDSqZ04FROn3du4CK2A,GjjSC7aAXdff8jERc3URYw,4',
 u'-3gIDSqZ04FROn3du4CK2A,ZHQU79PMUGCX0fEoyCcWlw,5']

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
hasattr(training_data,"toDF")

True

In [6]:
header = training_data.first()
header_test = test_data.first()

train_rdd = training_data.filter(lambda z: z != header) \
    .map(lambda x: x.split(',')).map(lambda x: (x[0], x[1], x[2]))

test_rdd = test_data.filter(lambda z: z != header) \
    .map(lambda x: x.split(',')).map(lambda x: (x[0], x[1], x[2]))


In [7]:
test_rdd.take(2)

[(u'user_id', u'business_id', u'rating'),
 (u'-3gIDSqZ04FROn3du4CK2A', u'-xIrVu_7prj_0GDl1tds1w', u'5')]

In [8]:
all_dist_busId_test = test_rdd.map(lambda x: x[1]).distinct()
all_dist_busId_train = train_rdd.map(lambda x: x[1]).distinct()

all_dist_busId_final = all_dist_busId_test.union(all_dist_busId_train).distinct()

In [9]:
all_dist_busId_final.count()

3524

In [10]:
all_busId_dict = {}
uniqueId_forAll_busId = all_dist_busId_final.zipWithUniqueId()
all_busId_dict = uniqueId_forAll_busId.collectAsMap()

In [11]:
all_dist_userId_test = test_rdd.map(lambda x: x[0]).distinct()
all_dist_userId_train = train_rdd.map(lambda x: x[0]).distinct()

all_dist_userId_final = all_dist_userId_test.union(all_dist_userId_train).distinct()

all_userId_dict = {}
uniqueId_forAll_userId = all_dist_userId_final.zipWithUniqueId()
all_userId_dict = uniqueId_forAll_userId.collectAsMap()


In [12]:
final_train_rdd2 = train_rdd.map(lambda x: ((all_userId_dict[x[0]], all_busId_dict[x[1]], x[2])))




In [13]:
final_train_rdd2.take(4)

[(76795, 1646, u'5'),
 (81588, 1035, u'5'),
 (15969, 2606, u'5'),
 (56804, 1583, u'4')]

In [14]:
test_rdd.take(3)

[(u'user_id', u'business_id', u'rating'),
 (u'-3gIDSqZ04FROn3du4CK2A', u'-xIrVu_7prj_0GDl1tds1w', u'5'),
 (u'-3gIDSqZ04FROn3du4CK2A', u'Bjn-GQg6hoUOkew7tTPcLA', u'5')]

In [15]:
final_test_df = test_rdd.map(lambda x:((all_userId_dict[x[0]], all_busId_dict[x[1]], x[2])))#.toDF(["user_id,business_id,rating"])
final_test_df.take(2)



[(41514, 2653, u'rating'), (56804, 3163, u'5')]

In [16]:
final_test_df = final_test_df.toDF(["user_id","business_id","rating"])

In [17]:
final_test_df.show(4)

+-------+-----------+------+
|user_id|business_id|rating|
+-------+-----------+------+
|  41514|       2653|rating|
|  56804|       3163|     5|
|  56804|       2903|     5|
|  56804|       1992|     4|
+-------+-----------+------+
only showing top 4 rows



In [18]:
final_test_pd_df = final_test_df.toPandas()

In [19]:
X_test = final_test_pd_df.iloc[:,:-1].values
Y_test = final_test_pd_df.iloc[:,2].values



In [20]:
final_train_df = final_train_rdd2.toDF(["user_id","business_id","rating"])

In [21]:
final_train_df.show(5)

+-------+-----------+------+
|user_id|business_id|rating|
+-------+-----------+------+
|  76795|       1646|     5|
|  81588|       1035|     5|
|  15969|       2606|     5|
|  56804|       1583|     4|
|  56804|       2207|     5|
+-------+-----------+------+
only showing top 5 rows



In [22]:
final_train_pd_df = final_train_df.toPandas()

In [23]:
X_train = final_train_pd_df.iloc[:,:-1].values

In [24]:
Y_train = final_train_pd_df.iloc[:,2].values

In [25]:

scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [26]:
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=10)  
classifier.fit(X_train, Y_train)  

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [27]:
Y_pred_test = classifier.predict(X_test)  

In [28]:
Y_pred_test

array([u'5', u'5', u'5', ..., u'5', u'5', u'4'], dtype=object)

In [29]:
from sklearn.metrics import classification_report, confusion_matrix  

In [31]:
print(confusion_matrix(Y_test, Y_pred_test))

[[  650   119   142   744  3711     0]
 [  276    70    88   413  1958     0]
 [  334    81   107   532  2479     0]
 [  644   156   226  1081  5220     0]
 [ 1528   306   446  2296 12481     0]
 [    0     0     0     0     1     0]]


In [32]:
print(classification_report(Y_test, Y_pred_test)) 

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           1       0.19      0.12      0.15      5366
           2       0.10      0.02      0.04      2805
           3       0.11      0.03      0.05      3533
           4       0.21      0.15      0.17      7327
           5       0.48      0.73      0.58     17057
      rating       0.00      0.00      0.00         1

   micro avg       0.40      0.40      0.40     36089
   macro avg       0.18      0.18      0.17     36089
weighted avg       0.32      0.40      0.34     36089

